In [4]:
## Step 1: Environment Setup
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

All libraries installed successfully!


In [ ]:
## Step 2: Data Loading and Processing
# Define the function to load JSON file into DataFrame chunks
def load_motion_data(json_file, chunk_size=100):
    with open(json_file, 'r') as f:
        data = json.load(f)

    frames = []
    if isinstance(data, list) and isinstance(data[0], list):
        for nested_list in data:  # First-level list
            for idx, entry in enumerate(nested_list):  # Second-level list
                if idx % chunk_size == 0 and idx > 0:
                    yield pd.DataFrame(frames)  # Yield the chunk
                    frames = []  # Reset frames
                if 'keypoints' in entry:
                    keypoints = entry['keypoints']
                    frame_data = {
                        f"keypoint_{i}_x": point[0] for i, point in enumerate(keypoints)
                    }
                    frame_data.update({
                        f"keypoint_{i}_y": point[1] for i, point in enumerate(keypoints)
                    })
                    frame_data['timestamp'] = len(frames)
                    frames.append(frame_data)
            # Yield remaining frames after loop
            if frames:
                yield pd.DataFrame(frames)
    else:
        print(f"Unsupported structure in {json_file}.")

# Define the directory containing the squash-processed JSON files
squash_data_dir = '/Users/ishaingersol/Desktop/Dataset/squash-processed/'
output_dir = '/Users/ishaingersol/Desktop/Dataset/processed_output/'  # Directory for intermediate outputs
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

# Function to process a single JSON file in chunks and save the results
def process_and_save_file_in_chunks(json_file, output_dir):
    file_path = os.path.join(squash_data_dir, json_file)
    chunk_idx = 0
    for chunk in load_motion_data(file_path):
        output_path = os.path.join(output_dir, f"{json_file.replace('.json', '')}_chunk_{chunk_idx}.csv")
        chunk.to_csv(output_path, index=False)
        print(f"Processed and saved chunk: {output_path}")
        chunk_idx += 1

# Process each file individually and save in chunks
squash_data_files = [f for f in os.listdir(squash_data_dir) if f.endswith('.json')]
print("Found JSON files:", squash_data_files)

for json_file in squash_data_files:
    process_and_save_file_in_chunks(json_file, output_dir)

print("All files processed and saved.")

# Combine all processed chunks into a single DataFrame if needed
all_motion_data = pd.concat(
    [pd.read_csv(os.path.join(output_dir, f)) for f in os.listdir(output_dir) if f.endswith('.csv')],
    ignore_index=True
)

# Display the shape of the DataFrame to confirm data loading
print(f"Loaded DataFrame Shape: {all_motion_data.shape}")
print("Sample DataFrame:")
print(all_motion_data.head())

In [ ]:
## Step 3: Keypoints Mapping
# Adjust keypoint indices based on supervisor's note
keypoint_mapping = {
    0: "Nose", 1: "Left Eye Inner", 2: "Left Eye", 3: "Left Eye Outer", 4: "Right Eye Inner",
    5: "Right Eye", 6: "Right Eye Outer", 7: "Left Ear", 8: "Right Ear", 9: "Mouth Left",
    10: "Mouth Right", 11: "Left Shoulder", 12: "Right Shoulder", 13: "Left Elbow", 14: "Right Elbow",
    15: "Left Wrist", 16: "Right Wrist", 17: "Left Pinky", 18: "Right Pinky", 19: "Left Index",
    20: "Right Index", 21: "Left Thumb", 22: "Right Thumb", 23: "Left Hip", 24: "Right Hip",
    25: "Left Knee", 26: "Right Knee", 27: "Left Ankle", 28: "Right Ankle", 29: "Left Heel",
    30: "Right Heel", 31: "Left Foot Index", 32: "Right Foot Index"
}

# Map additional keypoints up to 132 if needed
for idx in range(33, 133):
    keypoint_mapping[idx] = f"Keypoint_{idx}"

# Filter only columns that exist in the DataFrame
available_keypoint_columns = [col for col in [f"keypoint_{idx}_x" for idx in keypoint_mapping.keys()] + [f"keypoint_{idx}_y" for idx in keypoint_mapping.keys()] if col in all_motion_data.columns]
if 'timestamp' in all_motion_data.columns:
    available_keypoint_columns.append('timestamp')

# Extract available keypoint data for visualisation
keypoint_data = all_motion_data[available_keypoint_columns]

# Rename columns for clarity
keypoint_data.rename(columns={
    f"keypoint_{idx}_x": f"{name}_x" for idx, name in keypoint_mapping.items() if f"keypoint_{idx}_x" in keypoint_data.columns
}, inplace=True)
keypoint_data.rename(columns={
    f"keypoint_{idx}_y": f"{name}_y" for idx, name in keypoint_mapping.items() if f"keypoint_{idx}_y" in keypoint_data.columns
}, inplace=True)

# Display sample of the extracted data
print("Keypoint Data Sample:")
print(keypoint_data.head())

In [ ]:
## Step 4: Visualisation
# Plot motion trajectories of specific keypoints over time
def plot_keypoint_motion(data, keypoint_name):
    plt.figure(figsize=(10, 6))
    plt.plot(data['timestamp'], data[keypoint_name], label=keypoint_name, marker='o')
    plt.title(f"Motion Trajectory of {keypoint_name}")
    plt.xlabel("Time (s)")
    plt.ylabel("Position Value")
    plt.legend()
    plt.grid()
    plt.show()

# Example: Plot motion of a specific keypoint
plot_keypoint_motion(keypoint_data, "Right Wrist_x")

In [ ]:
## Step 5: Basic Motion Analysis
# Calculate motion magnitude for a specific keypoint
def calculate_motion_magnitude(data, keypoint_name):
    data[f"{keypoint_name}_magnitude"] = data[keypoint_name].diff().abs()
    return data

# Example: Analyse motion magnitude of a specific keypoint
keypoint_data = calculate_motion_magnitude(keypoint_data, "Right Wrist_x")
print("Motion Magnitude Analysis Sample:")
print(keypoint_data.head())

# Additional Steps (to be extended as needed):
# - Compare motion data across different files
# - Identify patterns corresponding to backhand/forehand movements
# - Extend analysis for videos once linked to the motion data